# **Greedy Search**

✔ Her adımda bir sonraki kelime olarak en yüksek olasılığa sahip kelimeyi seçer.

✔ Her adımda en yüksek olasılıklı token’i seçerek metin üretir. Alternatif seçenekleri değerlendirmediği için bazen düşük çeşitlilikte ve tekrarlayan çıktılar üretebilir.


✔ It selects the word with the highest probability as the next word at each step.

✔ By choosing the most likely token at each step, it generates text. Since it does not consider alternative options, it may sometimes produce low-diversity and repetitive outputs.

In [4]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [5]:
import torch
import numpy as np

def get_log_prob(logits, token_id):
    """
    Compute the log probability of a token.

    Args:
        logits (Tensor): Model output logits.
        token_id (Tensor): Selected token index.

    Returns:
        float: Log probability of the token.
    """
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    log_probabilities = torch.log(probabilities)
    return log_probabilities[token_id].item()

def greedy_search(input_ids, length=5):
    """
    Generate text using Greedy Search.

    Args:
        input_ids (Tensor): Initial token IDs.
        length (int): Maximum number of tokens to generate.

    Returns:
        Tensor: Generated token IDs.
    """
    generated_ids = input_ids.clone()

    for _ in range(length):
        outputs = model(generated_ids.to(device))
        logits = outputs.logits[:, -1, :]

        token_id = torch.argmax(logits, dim=-1)
        token_score = get_log_prob(logits.squeeze(), token_id)

        print(f"Token: {tokenizer.decode(token_id)} - Probability: {np.exp(token_score) * 100:.2f}%")

        generated_ids = torch.cat([generated_ids, token_id.unsqueeze(0)], dim=-1)

    return generated_ids

start_text = "I have a"
input_ids = tokenizer.encode(start_text, return_tensors="pt").to(device)

output_ids = greedy_search(input_ids, length=10)
output_text = tokenizer.decode(output_ids.squeeze().tolist(), skip_special_tokens=True)
print(f"\nGenerated text: {output_text}")

Token:  lot - Probability: 6.37%
Token:  of - Probability: 84.72%
Token:  respect - Probability: 7.25%
Token:  for - Probability: 93.29%
Token:  the - Probability: 16.37%
Token:  people - Probability: 7.61%
Token:  who - Probability: 29.16%
Token:  are - Probability: 12.84%
Token:  here - Probability: 14.37%
Token: . - Probability: 16.26%

Generated text: I have a lot of respect for the people who are here.
